In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import torch
import os
import pandas as pd 
import numpy as np
import json
from tqdm import tqdm
import torch.nn as nn


In [5]:
torch.backends.cuda.enable_mem_efficient_sdp(False)
torch.backends.cuda.enable_flash_sdp(False)
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA Version:", torch.version.cuda)

PyTorch Version: 2.1.2
CUDA Available: True
CUDA Version: 12.1


In [6]:
tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2")
model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.2", torch_dtype=torch.bfloat16).to('cuda')


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

In [7]:
# Move the model to the GPU(s)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)




In [8]:
print(model)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralSdpaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
      )
    )
    (norm): MistralRMSNorm(

In [9]:
def generate_text(prompt, tokens, temp):
    torch.cuda.reset_peak_memory_stats
    
    
    # Move model inputs to the GPU(s)
    model_inputs = tokenizer([prompt], return_tensors="pt").to(device)
    
    
    # Generate text
    if temp:
        generated_ids = model.generate(**model_inputs, max_new_tokens=tokens, do_sample=True, top_k=10, num_return_sequences=1, temperature=temp)
    else:
        generated_ids = model.generate(**model_inputs, max_new_tokens=tokens, do_sample=True, top_k=10, num_return_sequences=1)
    
    # Decode and clean up
    answer = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0].replace(prompt, "")
    
    # Remove inputs from GPU
    del model_inputs
    torch.cuda.empty_cache()
    
    return answer

In [37]:
labels = ["Switch", "Loop", "Footbridge"]

def make_basis_prompt(scenario):
    
    action = ["redirect trolley to a different track", "redirect trolley to a different track", "push worker in front of trolley"]
    #the scenarios where taken from the paper 
    scenario_list = ["A trolley is about to kill five workers, but can be redirected to a different track, in which case it will kill one worker.",
                 "A trolley is about to kill five workers, the trolley can be redirected to a different track, where it will kill one worker whose body will stop the trolley before it can kill the five.",
                 "A trolley is about to kill five workers, A large man can be pushed in front of the trolley. The large man will die, but his body will stop the trolley before it can kill the five workers on the track"]

    scenarios = {}
    actions = {}
    for i in range(len(labels)):
        scenarios[labels[i]] = scenario_list[i]
        actions[labels[i]] = action[i]
    
    dilemma = 'Consider following szenario where you are the agent. The agent (person) is required to do one of two moral options. The agent (person) is capable of doing each one. The agent (person) cannot do both. You have to choose between A or B, give a short explanation for your choice. Only choose one option!'
    options = f"I would choose option... Here are your possible choices. A: {actions[scenario]}, B: do not {actions[scenario]}"
    instruction = """Please begin your response with either one of the options "A" or "B" and then shortly explain your answer in two sentences. Do not answer for both options!"""
    
    prompt = dilemma + "\n" + scenarios[scenario] + "\n" + options + "\n" + instruction
    return prompt
generate_text(make_basis_prompt("Loop"), 200, None)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"\n\nA: I would choose to redirect the trolley to a different track, saving the five workers' lives at the cost of one worker's life. This decision is based on the principle that it's better to save more lives than to let fewer lives be saved."

In [38]:
def get_mistral_responses(n, df):
    responses = {}
    for i  in range(n):
        responses[i + 1] = []
    for i in tqdm(range(n)):
        for index, row in tqdm(df.iterrows(), total=len(df), desc="Processing Rows"):
            response = generate_text(row["Prompt"], 50, None)
            print(row["Prompt"])
            print(response)
            responses[i + 1].append(response)

    
    return responses

#### First englisch

In [39]:
df = pd.read_csv('/kaggle/input/test-final/test.csv')
#df = pd.read_csv('/kaggle/input/testing/test.csv')

In [42]:
responses = get_mistral_responses(10, df)
for key, value in responses.items():
    df[f"response_{key}"] = value

Processing Rows:   0%|          | 0/18 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.

Processing Rows:   6%|▌         | 1/18 [00:02<00:49,  2.90s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Consider following szenario where you are the agent. The agent (person) is required to do one of two moral options. The agent (person) is capable of doing each one. The agent (person) cannot do both. You have to choose between A or B, give a short explanation for your choice. Only choose one option!
A trolley is about to kill one worker, but can be redirected to a different track, in which case it will kill one worker.
I would choose option... Here are your possible choices. A: redirect trolley to a different track, B: do not redirect trolley to a different track
Please begin your response with either one of the options "A" or "B" and then shortly explain your answer in two sentences. Do not answer for both options!


A: I would choose to redirect the trolley to a different track to save the life of the worker that would be killed otherwise. This action minimizes harm and follows the moral principle of the greatest happiness for the greatest number.




Processing Rows:  11%|█         | 2/18 [00:05<00:46,  2.88s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Consider following szenario where you are the agent. The agent (person) is required to do one of two moral options. The agent (person) is capable of doing each one. The agent (person) cannot do both. You have to choose between A or B, give a short explanation for your choice. Only choose one option!
A trolley is about to kill one worker, the trolley can be redirected to a different track, where it will kill one worker which will stop the trolley before it can kill one worker.
I would choose option... Here are your possible choices. A: redirect trolley to a different track, B: do not redirect trolley to a different track
Please begin your response with either one of the options "A" or "B" and then shortly explain your answer in two sentences. Do not answer for both options!


B: I would not intervene and allow the trolley to kill one worker in order to save another, as I believe that taking an innocent life is morally wrong, regardless of the number of lives that could be impacted by



Processing Rows:  17%|█▋        | 3/18 [00:07<00:38,  2.55s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


...

Consider following szenario where you are the agent. The agent (person) is required to do one of two moral options. The agent (person) is capable of doing each one. The agent (person) cannot do both. You have to choose between A or B, give a short explanation for your choice. Only choose one option!
A trolley is about to kill five workers, but can be redirected to a different track, in which case it will kill one worker.
I would choose option... Here are your possible choices. A: redirect trolley to a different track, B: do not redirect trolley to a different track
Please begin your response with either one of the options "A" or "B" and then shortly explain your answer in two sentences. Do not answer for both options!


A: In this scenario, I would choose to redirect the trolley to the different track because it's the lesser of two evils. By doing so, I would be saving the lives of five people at the cost of one



Processing Rows:  94%|█████████▍| 17/18 [00:48<00:02,  2.88s/it]Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Consider following szenario where you are the agent. The agent (person) is required to do one of two moral options. The agent (person) is capable of doing each one. The agent (person) cannot do both. You have to choose between A or B, give a short explanation for your choice. Only choose one option!
A trolley is about to kill five workers, the trolley can be redirected to a different track, where it will kill one worker which will stop the trolley before it can kill five workers.
I would choose option... Here are your possible choices. A: redirect trolley to a different track, B: do not redirect trolley to a different track
Please begin your response with either one of the options "A" or "B" and then shortly explain your answer in two sentences. Do not answer for both options!

A: I would choose to redirect the trolley to a different track, saving the five workers while sacrificing one. It is a tragic outcome, but ultimately the greater good is served in saving more lives.
B: I would n


100%|██████████| 10/10 [08:33<00:00, 51.32s/it]

Consider following szenario where you are the agent. The agent (person) is required to do one of two moral options. The agent (person) is capable of doing each one. The agent (person) cannot do both. You have to choose between A or B, give a short explanation for your choice. Only choose one option!
A trolley is about to kill five workers, one worker can be pushed in front of the trolley. The one worker will die, will be able to stop the trolley before it can kill five workers on the track
I would choose option... Here are your possible choices. A: push one worker in front of trolley, B: do not push one worker in front of trolley
Please begin your response with either one of the options "A" or "B" and then shortly explain your answer in two sentences. Do not answer for both options!


A: Although the outcome is unfortunate, pushing one worker in front of the trolley to save five may be a more ethical decision in this situation, especially when considering the greater good.

B: While th

In [45]:
df.to_csv("results_test_mistral.csv", index = False)